import libraries(I provide all libs that I need when make this tasks, if you need some external import them here)

In [33]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col
from pyspark.sql.functions import max, avg, min
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from pyspark.sql.functions import when
from distutils.version import LooseVersion
import pandas as pd

create local SparkSession

In [34]:
spark = SparkSession.builder.master("local[*]").appName("FirstSparkApp").getOrCreate()
spark

read csv with inferschema

In [35]:
df1 = spark.read.option("inferSchema",True).option("header",True).csv("ds_salaries.csv")


read csv one more time with the same code and you will see that it almostly don't take time, because info already in SparkSession and it will not read nothing
from this file

In [36]:
df1 = spark.read.option("inferSchema",True).option("header",True).csv("ds_salaries.csv")

write schema of scv on screen

In [37]:
df1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



create schema of this scv

In [38]:
schema = StructType([ \
    StructField("id",IntegerType(),True), \
    StructField("work_year",IntegerType(),True), \
    StructField("experience_level",StringType(),True), \
    StructField("employment_type", StringType(), True), \
    StructField("job_title", StringType(), True), \
    StructField("salary", IntegerType(), True), \
    StructField("salary_currency", StringType(), True), \
    StructField("salary_in_usd", IntegerType(), True), \
    StructField("employee_residence", StringType(), True), \
    StructField("remote_ratio", IntegerType(), True), \
    StructField("company_location", StringType(), True), \
    StructField("company_size", StringType(), True) \
  ])

restart kernel without cleaning output and after restarting you need to initialize SparkSession, after initialize start execute only cells from cell with schema=
=StructType.... 
To restart kernel click Kernel, Restart.

read ds_salaries with predefined schema and compare results from this cell and cell with inferSchema

In [39]:
spark = SparkSession.builder.master("local[*]").appName("FirstSparkApp").getOrCreate()
df2 = spark.read.schema(schema).option("header",True).csv("ds_salaries.csv")

this happens because read operation is lazy(transformation), but if you use inferschema it start to be action that will create Spark Job, because Spark need to loop throw all file to check datatypes for all columns and this can harm to your code(if we compare to parquet, it will also go to check data types, but parquet provide meta information, so Spark will not go throw all file, he will just read meta information, but csv don't provide such meta information). Also header make Spark to create one more Spark Job to check first line
to define name of columns and remember to skeep it when reading. Actual reading start when you will use first action. More about Spark Jobs you will see in next topic

write schema of scv on screen one more time and compare with previous

In [40]:
df2.printSchema()

root
 |-- id: integer (nullable = true)
 |-- work_year: integer (nullable = true)
 |-- experience_level: string (nullable = true)
 |-- employment_type: string (nullable = true)
 |-- job_title: string (nullable = true)
 |-- salary: integer (nullable = true)
 |-- salary_currency: string (nullable = true)
 |-- salary_in_usd: integer (nullable = true)
 |-- employee_residence: string (nullable = true)
 |-- remote_ratio: integer (nullable = true)
 |-- company_location: string (nullable = true)
 |-- company_size: string (nullable = true)



now continue to work with one of the dataframes that you create

print data in dataframe using df.show

In [41]:
df2.show()

+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
| id|work_year|experience_level|employment_type|           job_title|  salary|salary_currency|salary_in_usd|employee_residence|remote_ratio|company_location|company_size|
+---+---------+----------------+---------------+--------------------+--------+---------------+-------------+------------------+------------+----------------+------------+
|  0|     2020|              MI|             FT|      Data Scientist|   70000|            EUR|        79833|                DE|           0|              DE|           L|
|  1|     2020|              SE|             FT|Machine Learning ...|  260000|            USD|       260000|                JP|           0|              JP|           S|
|  2|     2020|              SE|             FT|   Big Data Engineer|   85000|            GBP|       109024|                GB|          50|     

print data in dataframe using display(df.toPandas())

In [42]:
display(df2.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...,...
602,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


create df_job_title that consists from all job_titles without duplicates

In [43]:
df_job_title = df2.select(col("job_title")).dropDuplicates()

print all rows from df_job_titles without truncating jobs

In [44]:
df_job_title.show(truncate = False)

+----------------------------------------+
|job_title                               |
+----------------------------------------+
|3D Computer Vision Researcher           |
|Lead Data Engineer                      |
|Head of Machine Learning                |
|Data Specialist                         |
|Data Analytics Lead                     |
|Machine Learning Scientist              |
|Lead Data Analyst                       |
|Data Engineering Manager                |
|Staff Data Scientist                    |
|ETL Developer                           |
|Director of Data Engineering            |
|Product Data Analyst                    |
|Principal Data Scientist                |
|AI Scientist                            |
|Director of Data Science                |
|Machine Learning Engineer               |
|Lead Data Scientist                     |
|Machine Learning Infrastructure Engineer|
|Data Science Engineer                   |
|Machine Learning Manager                |
+----------

create  df_analytic that will consists from max, avg, min USD salaries for all job_titles using groupBy. name of fields is avg_salary, min_salary, max_salary

In [45]:
df_analytic = df2.groupBy("job_title") \
                    .agg(max("salary_in_usd").alias("max_salary"), \
                         avg("salary_in_usd").alias("avg_salary"), \
                         min("salary_in_usd").alias("min_salary"))

print all rows from df_analytic without trancating jobs

In [46]:
df_analytic.show(truncate=False)

+----------------------------------------+----------+------------------+----------+
|job_title                               |max_salary|avg_salary        |min_salary|
+----------------------------------------+----------+------------------+----------+
|3D Computer Vision Researcher           |5409      |5409.0            |5409      |
|Lead Data Engineer                      |276000    |139724.5          |56000     |
|Head of Machine Learning                |79039     |79039.0           |79039     |
|Data Specialist                         |165000    |165000.0          |165000    |
|Data Analytics Lead                     |405000    |405000.0          |405000    |
|Machine Learning Scientist              |260000    |158412.5          |12000     |
|Lead Data Analyst                       |170000    |92203.0           |19609     |
|Data Engineering Manager                |174000    |123227.2          |59303     |
|Staff Data Scientist                    |105000    |105000.0          |1050

now you need to add in df_analytic column row_id, that will show order of all job_titles depending on avg salary. they should be descending

In [47]:
windowSpec = Window.orderBy(col("avg_salary").desc())
df_analytic = df_analytic.withColumn("row_id",row_number().over(windowSpec))

print all data from df_analytic

In [48]:
display(df_analytic.toPandas())

,job_title,max_salary,avg_salary,min_salary,row_id
0,Data Analytics Lead,405000,405000.000000,405000,1
1,Principal Data Engineer,600000,328333.333333,185000,2
2,Financial Data Analyst,450000,275000.000000,100000,3
3,Principal Data Scientist,416000,215242.428571,148261,4
4,Director of Data Science,325000,195074.000000,130026,5
5,Data Architect,266400,177873.909091,90700,6
6,Applied Data Scientist,380000,175655.000000,54238,7
7,Analytics Engineer,205300,175000.000000,135000,8
8,Data Specialist,165000,165000.000000,165000,9
9,Head of Data,235000,160162.600000,32974,10


it isn't beautifull, so we need to put now row_id on first place in df_analytic

In [49]:
df_analytic = df_analytic.selectExpr("row_id", "max_salary", "min_salary", "avg_salary")

print df_analytic now

In [50]:
display(df_analytic.toPandas())

,row_id,max_salary,min_salary,avg_salary
0,1,405000,405000,405000.000000
1,2,600000,185000,328333.333333
2,3,450000,100000,275000.000000
3,4,416000,148261,215242.428571
4,5,325000,130026,195074.000000
5,6,266400,90700,177873.909091
6,7,380000,54238,175655.000000
7,8,205300,135000,175000.000000
8,9,165000,165000,165000.000000
9,10,235000,32974,160162.600000


here you need to create df_exp_lvl with the biggest usd_salary(biggest_salary) for each experience_level(you need to save all fields like in entire dataframe)

In [51]:
windowSpec = Window.partitionBy("experience_level")
df_exp_lvl = df2.withColumn("biggest_salary",max("salary_in_usd").over(windowSpec))

print here df_exp_lvl

In [52]:
display(df_exp_lvl.toPandas())

,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size,biggest_salary
0,5,2020,EN,FT,Data Analyst,72000,USD,72000,US,100,US,L,250000
1,10,2020,EN,FT,Data Scientist,45000,EUR,51321,FR,0,FR,S,250000
2,12,2020,EN,FT,Data Scientist,35000,EUR,39916,FR,0,FR,M,250000
3,16,2020,EN,FT,Data Engineer,4450000,JPY,41689,JP,100,JP,S,250000
4,18,2020,EN,FT,Data Science Consultant,423000,INR,5707,IN,50,IN,M,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,597,2022,SE,FT,Data Analyst,170000,USD,170000,US,100,US,M,412000
603,602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M,412000
604,603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M,412000
605,604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M,412000


create df_best that consists from rows where salary of guy same as biggest salary for other people in his exp_lvl and choose only columns: id, experience_level, biggest_salary, employee_residence

In [53]:
df_best = df_exp_lvl.select("id", "experience_level", "biggest_salary", "employee_residence").where(df_exp_lvl["salary_in_usd"] == df_exp_lvl["biggest_salary"])

print df_best

In [54]:
display(df_best.toPandas())

,id,experience_level,biggest_salary,employee_residence
0,37,EN,250000,US
1,252,EX,600000,US
2,33,MI,450000,US
3,97,MI,450000,US
4,63,SE,412000,US


drop duplicates if exist by experience_level

In [55]:
df_best = df_best.dropDuplicates(["experience_level"])

print df_best

In [56]:
display(df_best.toPandas())

,id,experience_level,biggest_salary,employee_residence
0,37,EN,250000,US
1,252,EX,600000,US
2,33,MI,450000,US
3,63,SE,412000,US


create df_new_best from df_best without id, and make the next: when exp_level = MI we want middle, when SE we want senior, else Null

In [57]:
df_new_best = df_best.drop("id").withColumn("experience_level", when(df_best.experience_level == "MI","middle")
                                                                .when(df_best.experience_level == "SE","senior"))


print df_new_best

In [58]:
display(df_new_best.toPandas())

,experience_level,biggest_salary,employee_residence
0,None,250000,US
1,None,600000,US
2,middle,450000,US
3,senior,412000,US


write df_new_best like 1.csv and load then it to df_final

In [59]:
df_new_best.toPandas().to_csv('1.csv')
df_final = spark.read.option("inferSchema",True).option("header",True).csv("1.csv")

print df_final

In [60]:
display(df_final.toPandas())

,_c0,experience_level,biggest_salary,employee_residence
0,0,None,250000,US
1,1,None,600000,US
2,2,middle,450000,US
3,3,senior,412000,US


filter df_final to delete experience_level where it Null, then join this table by biggest_salary(salary_in_usd) and employee_residence with entire df

In [61]:
df_final = df_final.where(col("experience_level").isNotNull()).join(df2,(df_final.biggest_salary == df2.salary_in_usd) & (df_final.employee_residence == df2.employee_residence),"inner")

print df_final

In [62]:
display(df_final.toPandas())

,_c0,experience_level,biggest_salary,employee_residence,id,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2,middle,450000,US,33,2020,MI,FT,Research Scientist,450000,USD,450000,US,0,US,M
1,3,senior,412000,US,63,2020,SE,FT,Data Scientist,412000,USD,412000,US,100,US,L
2,2,middle,450000,US,97,2021,MI,FT,Financial Data Analyst,450000,USD,450000,US,100,US,L


last task is to save in variable and then print this variable of the biggest salary_in_usd from df_final

In [66]:
max_salary = df_final.agg(max('salary_in_usd').alias('max_salary_in_usd')).collect()[0]['max_salary_in_usd']
highest_salary_rows = df_final.filter(df_final['salary_in_usd'] == max_salary).collect()
for row in highest_salary_rows:
    print(row)

Row(_c0=2, experience_level='middle', biggest_salary=450000, employee_residence='US', id=33, work_year=2020, experience_level='MI', employment_type='FT', job_title='Research Scientist', salary=450000, salary_currency='USD', salary_in_usd=450000, employee_residence='US', remote_ratio=0, company_location='US', company_size='M')
Row(_c0=2, experience_level='middle', biggest_salary=450000, employee_residence='US', id=97, work_year=2021, experience_level='MI', employment_type='FT', job_title='Financial Data Analyst', salary=450000, salary_currency='USD', salary_in_usd=450000, employee_residence='US', remote_ratio=100, company_location='US', company_size='L')


It is the end of PySpark basics. In other lessons you will learn optimizations technics and how to make distributed system